<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Interactive-Visualization-Lab" data-toc-modified-id="Interactive-Visualization-Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Interactive Visualization Lab</a></span><ul class="toc-item"><li><span><a href="#1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011." data-toc-modified-id="1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.</a></span></li><li><span><a href="#2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011." data-toc-modified-id="2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.</a></span></li><li><span><a href="#3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories)." data-toc-modified-id="3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories).-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).</a></span></li><li><span><a href="#4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands." data-toc-modified-id="4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.</a></span></li><li><span><a href="#5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below." data-toc-modified-id="5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below.-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.</a></span></li><li><span><a href="#6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart." data-toc-modified-id="6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart.-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.</a></span></li><li><span><a href="#7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges." data-toc-modified-id="7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges.-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.</a></span></li><li><span><a href="#8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description." data-toc-modified-id="8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description.-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.</a></span></li></ul></li></ul></div>

# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [4]:
import pandas as pd
import plotly.offline as py
import cufflinks as cf
from ipywidgets import interact
import plotly.graph_objs as go
cf.go_offline()

In [6]:
data = pd.read_excel('../data/Online Retail.xlsx')
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [7]:
data1 = data[(data['Country']!='United Kingdom')  & (data['InvoiceDate'] >= '2011-04-01') & 
             (data['InvoiceDate'] <= '2011-04-30')]
data1 = data1.pivot_table(values=['Quantity','Revenue'], index='Country', aggfunc='sum')
data1 = data1.reset_index()

In [10]:
data1.iplot(kind='bar', x='Country', xTitle='Country',yTitle='Total', 
            title='Quantity / Revenue by Country (excluded U.K.) Apr 2011')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [8]:
data2 = data[(data['Country']=='France')  & (data['InvoiceDate'] >= '2011-01-01 00:00:00') & 
             (data['InvoiceDate'] <= '2011-05-31 23:59:59')]
data2 = data2.groupby('InvoiceDate')[['Quantity','Revenue']].sum()
data2 = data2.reset_index()

In [9]:
data2.iplot(kind='line', x='InvoiceDate', xTitle='Date',yTitle='Quantity', 
            title='quantity/revenue sold to France from Jan 1st to May 31st 2011')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [12]:
data3 = data[['Quantity','UnitPrice','Country']][data['Description']=='PARTY BUNTING']
data3 = data3.groupby('Country')[['Quantity','UnitPrice']].mean()
data3 = data3.reset_index()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [13]:
selection = ['EIRE', 'Germany', 'France', 'Netherlands']
data4 = data[data['Country'].isin(selection)]
data4 = data4.pivot_table(values='InvoiceNo', index='Quantity', 
                          columns='Country', aggfunc='count')

In [15]:
data4.iplot(kind='hist', xTitle='Quantity', subplots=True,
yTitle='Num', title='Distributions of quantity x invoice')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [16]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [18]:
data5 = data[(data['Description'].isin(product_list)) & (data['Country'].isin(country_list))]
data5 = data5.pivot_table(values='Revenue', index='Country', 
                          columns='Description', aggfunc='sum')

In [19]:
data5

Description,CREAM HANGING HEART T-LIGHT HOLDER,JUMBO BAG RED RETROSPOT,REGENCY CAKESTAND 3 TIER
Country,,,
EIRE,2740.80,278.72,7388.55
France,131.75,903.37,2816.85
Germany,35.40,1072.76,9061.95
Netherlands,1167.00,3468.00,3166.35


In [20]:
data5.iplot(kind='bar', subplots=True, subplot_titles=['CREAM HOLDER','JUMBO BAG','CAKESTAND'], 
            shape=(1, 3),legend=False, 
            title='Country Revenue of the products')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [21]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [23]:
@interact(Selection=['Day','Month','Year'])

def linechart(Selection):
    data6 = uk.pivot_table(values='Quantity', index=Selection,
                            aggfunc='sum').reset_index()

    data6.iplot(kind='line', x=Selection, xTitle=Selection.title(), 
               yTitle='Quantity Sold', title='Quantity Sold by ' + Selection.title())
#importante usar el @interact sino no funciona

interactive(children=(Dropdown(description='Selection', options=('Day', 'Month', 'Year'), value='Day'), Output…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [25]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

In [30]:
products = products.head()
products

,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
Description,,,,,
4 PURPLE FLOCK DINNER CANDLES,35,134,2.318421,255.46,30
50'S CHRISTMAS GIFT BAG LARGE,100,1721,1.247900,2067.25,98
DOLLY GIRL BEAKER,100,661,1.250000,826.25,77
I LOVE LONDON MINI BACKPACK,55,181,4.150000,751.15,46
NINE DRAWER OFFICE TIDY,25,44,14.761538,628.40,24


In [31]:
invoices=len(products['InvoiceNo'].unique())
customers=len(products['CustomerID'].unique())

In [32]:
#defino funcion, tarda mucho cada plot
def Scatter(invoices,customers):
    layout = go.Layout(xaxis=dict(range=[0, invoices]),
                       yaxis=dict(range=[0, customers])
    products.iplot(x='InvoiceNo', y='CustomerID', categories='Description', 
                   xTitle='Customers',yTitle='Invoices', layout = layout)

SyntaxError: invalid syntax (<ipython-input-32-c58307426c49>, line 5)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [ ]:
#sin tiempo para hacerlo